In [5]:
!pip install pymupdf transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 36.0 MB/s eta 0:00:00


In [6]:
import fitz  # PyMuPDF

def read_pdf(file_path):
    text = ""
    pdf_document = fitz.open(file_path)
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text

# Leer archivo de ejemplo
text = read_pdf('CV1.pdf')
print(text)

F U L L S T A C K  D E V E L O P E R
P U J I  J U N A E D I
M O H
P R O F I L E  S U M M A R Y
I am Junaedi, a programmer who has experience working in the field of software development for
more than 6 years. have good reasoning and problem solving and interested in developing my
skills and career in a company that develops applications with AI and machine learning in order to
compete with other companies
T E C H N I C A L  S K I L L
E D U C A T I O N
Ahli Muda, information management
State Polytechnic of Malang, Malang
Jun, 2016 - Apr, 2018
analysis
Development
Tools
Server
Communication
Adaptation
Indonesian
English
Javanese language
Leadership
MySQL
MongoDB
PHP
Python
NodeJs
HTML
CSS
Java Script
GIT
Visual Studio Code
Word
Spread Sheet
Linux
Apache2
Nginx
Docker
SSH
Business Model
ERD
Flow Chart
API Design
Boj onegoro,  Jawa Ti mur +6282331472499 me@j unae. i d
/j unaedi puj i
Head of HMJ Information Management
Focus on programming
Have participated in several programming competitio

In [9]:
from transformers import pipeline
import re

# Configuración del modelo de NLP
nlp = pipeline("ner", model="bert-base-cased")

def extract_information(text):
    result = {
        "nombre_completo": None,
        "email": None,
        "telefono": None,
        "anos_experiencia": None,
        "formacion_IA": None,
        "scores": {
            "nombre_completo": 0.0,
            "email": 0.0,
            "telefono": 0.0,
            "anos_experiencia": 0.0,
            "formacion_IA": 0.0
        }
    }

    entities = nlp(text)
    for entity in entities:
        if entity['entity'] == 'B-PER':
            result["nombre_completo"] = entity['word']
            result["scores"]["nombre_completo"] = 0.9  # Dummy score

    email_match = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
    result["email"] = email_match.group() if email_match else None
    result["scores"]["email"] = 0.95 if email_match else 0.0

    phone_match = re.search(r'\b\d{10,}\b', text)
    result["telefono"] = phone_match.group() if phone_match else None
    result["scores"]["telefono"] = 0.95 if phone_match else 0.0

    experience_match = re.search(r'\d+', text)
    result["anos_experiencia"] = experience_match.group() if experience_match else None
    result["scores"]["anos_experiencia"] = 0.90 if experience_match else 0.0

    result["formacion_IA"] = 'S' if 'inteligencia artificial' in text.lower() else 'N'
    result["scores"]["formacion_IA"] = 0.85 if result["formacion_IA"] == 'S' else 0.0

    return result


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import json

file_names = ['CV1.pdf', 'CV2.pdf', 'CV3.pdf', 'CV4.pdf', 'CV5.pdf', 'CV6.pdf', 'CV7.pdf']
results = []

for file_name in file_names:
    text = read_pdf(file_name)
    info = extract_information(text)
    results.append(info)

# Guardar los resultados en un archivo JSON
with open('/content/resultados.json', 'w') as f:
    json.dump(results, f, indent=4)



In [12]:
!pip install spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 60.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
import spacy

# Comprobar la lista de modelos disponibles
spacy.info()


{'spacy_version': '3.7.6',
 'location': '/usr/local/lib/python3.10/dist-packages/spacy',
 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35',
 'python_version': '3.10.12',
 'pipelines': {'en_core_web_sm': '3.7.1'}}

In [14]:
import spacy
import re
import json

nlp = spacy.load("es_core_news_sm")

def extract_information(text):
    doc = nlp(text)
    result = {
        "nombre_completo": None,
        "email": None,
        "telefono": None,
        "anos_experiencia": None,
        "formacion_IA": None,
        "scores": {
            "nombre_completo": 0.0,
            "email": 0.0,
            "telefono": 0.0,
            "anos_experiencia": 0.0,
            "formacion_IA": 0.0
        }
    }

    # Nombre completo (considerando que es una entidad PERSON en español)
    for ent in doc.ents:
        if ent.label_ == 'PER':
            result["nombre_completo"] = ent.text
            result["scores"]["nombre_completo"] = 0.9  # Dummy score
            break

    # Email y teléfono
    email_match = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
    result["email"] = email_match.group() if email_match else None
    result["scores"]["email"] = 0.95 if email_match else 0.0

    phone_match = re.search(r'\b\d{10,}\b', text)
    result["telefono"] = phone_match.group() if phone_match else None
    result["scores"]["telefono"] = 0.95 if phone_match else 0.0

    # Años de experiencia
    experience_match = re.search(r'\d+', text)
    result["anos_experiencia"] = experience_match.group() if experience_match else None
    result["scores"]["anos_experiencia"] = 0.90 if experience_match else 0.0

    # Formación en IA
    result["formacion_IA"] = 'S' if 'inteligencia artificial' in text.lower() else 'N'
    result["scores"]["formacion_IA"] = 0.85 if result["formacion_IA"] == 'S' else 0.0

    return result


In [15]:
results = []

for file_name in file_names:
    text = read_pdf(file_name)
    info = extract_information(text)
    results.append(info)

# Guardar los resultados en un archivo JSON
with open('/content/resultados.json', 'w') as f:
    json.dump(results, f, indent=4)

# Descargar el archivo JSON
from google.colab import files
files.download('/content/resultados.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>